In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1588049897809_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
conf = SparkConf().setAppName("spamclassification").setMaster("spark://master:7077")
sc = SparkContext.getOrCreate()
spark = SparkSession.builder.config( conf = conf ).appName("spamclassification").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#defining the schema to help read the file
from pyspark.sql.types import StructField,StructType,StringType,FloatType,IntegerType,DoubleType,BooleanType

MySchema = StructType([
    StructField("business_id", StringType(), True),        
    StructField("name", StringType(), True),
    StructField("stars", StringType(), True),
    StructField("review_count", StringType(), True),
    StructField("review_id", StringType(), True),
    StructField("user_id", StringType(), True),
    StructField("review_stars", IntegerType(), True),
    StructField("useful", IntegerType(), True),
    StructField("funny", IntegerType(), True),
    StructField("cool", IntegerType(), True),
    StructField("text", StringType(), True),
    StructField("date", StringType(), True),
])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#reading the csv file got after preprocessing
inputdf = spark.read.option("header","true").schema(MySchema).csv("s3://reviwssannilbhav/reviews1.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
#displaying the dataframe read 
inputdf.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+------------+--------------------+--------------------+------------+------+-----+----+--------------------+--------------+
|         business_id|                name|stars|review_count|           review_id|             user_id|review_stars|useful|funny|cool|                text|          date|
+--------------------+--------------------+-----+------------+--------------------+--------------------+------------+------+-----+----+--------------------+--------------+
|cKyLV5oWZJ2NudWgq...|Oasis Auto Center...|  4.5|          38|edUqROoq7qAVzR2b2...|4oE6BjeHyTVRqd_82...|           2|     3|    0|   0|This place was mu...|9/4/2013 22:11|
+--------------------+--------------------+-----+------------+--------------------+--------------------+------------+------+-----+----+--------------------+--------------+
only showing top 1 row

In [6]:
# final dropping of null values if any 
updatedDF = inputdf.dropna()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Generating a sepcial Feature Id from user_id, review_date, business_id
def generateFeatureId (user_id, review_date, business_id):
    return user_id + "-" + review_date + "-" + business_id

from pyspark.sql.functions import udf
generateFeatureId_udf = udf(generateFeatureId, StringType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# applying the function to generate the id
from pyspark.sql.functions import col

from pyspark.sql.functions import udf

featureIdDF = updatedDF.withColumn("feature_id",generateFeatureId_udf(updatedDF["user_id"],updatedDF["date"],updatedDF["business_id"]))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
featureIdDF.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+------------+--------------------+--------------------+------------+------+-----+----+--------------------+--------------+--------------------+
|         business_id|                name|stars|review_count|           review_id|             user_id|review_stars|useful|funny|cool|                text|          date|          feature_id|
+--------------------+--------------------+-----+------------+--------------------+--------------------+------------+------+-----+----+--------------------+--------------+--------------------+
|cKyLV5oWZJ2NudWgq...|Oasis Auto Center...|  4.5|          38|edUqROoq7qAVzR2b2...|4oE6BjeHyTVRqd_82...|           2|     3|    0|   0|This place was mu...|9/4/2013 22:11|4oE6BjeHyTVRqd_82...|
+--------------------+--------------------+-----+------------+--------------------+--------------------+------------+------+-----+----+--------------------+--------------+--------------------+
only showing top 1 row

In [10]:
# Combine Useful, Funny, Cool into single column named - useful_review_count
def calculation_useful_review_count(useful, funny, cool):
    return useful + funny + cool;

calculation_useful_review_count_udf = udf(calculation_useful_review_count, IntegerType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
#applying function to combine all votes into one
combinedUsefulDF = featureIdDF.withColumn("useful_review_count",calculation_useful_review_count_udf(featureIdDF['useful'],featureIdDF["funny"],featureIdDF["cool"]))
combinedUsefulDF = combinedUsefulDF.drop("useful","funny","cool");
combinedUsefulDF.show(1);
combinedUsefulDF.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+------------+--------------------+--------------------+------------+--------------------+--------------+--------------------+-------------------+
|         business_id|                name|stars|review_count|           review_id|             user_id|review_stars|                text|          date|          feature_id|useful_review_count|
+--------------------+--------------------+-----+------------+--------------------+--------------------+------------+--------------------+--------------+--------------------+-------------------+
|cKyLV5oWZJ2NudWgq...|Oasis Auto Center...|  4.5|          38|edUqROoq7qAVzR2b2...|4oE6BjeHyTVRqd_82...|           2|This place was mu...|9/4/2013 22:11|4oE6BjeHyTVRqd_82...|                  3|
+--------------------+--------------------+-----+------------+--------------------+--------------------+------------+--------------------+--------------+--------------------+-------------------+
only showing top 1 row

D

In [12]:
#selecting the feature id and review text to get the sentiment score for a particular review
reviewsSummaryDF = combinedUsefulDF.select("feature_id","text")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
reviewsSummaryDF.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+
|          feature_id|                text|
+--------------------+--------------------+
|4oE6BjeHyTVRqd_82...|This place was mu...|
+--------------------+--------------------+
only showing top 1 row

In [14]:
#installing python libraries
sc.install_pypi_package("textblob")
# sc.install_pypi_package("pytest-benchmark")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
from textblob import TextBlob
# Sentiment Analysis, returns the sentiment score 
def getSentiment(text):
    sentiment = TextBlob(text).sentiment.polarity
    return sentiment

sentiment_score_udf = udf(getSentiment, DoubleType())


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
combinedUsefulDF = combinedUsefulDF.dropna()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
sentiment_score_df = combinedUsefulDF.withColumn("sentiment_score",sentiment_score_udf(combinedUsefulDF["text"]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
sentiment_score_df.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+------------+--------------------+--------------------+------------+--------------------+--------------+--------------------+-------------------+-------------------+
|         business_id|                name|stars|review_count|           review_id|             user_id|review_stars|                text|          date|          feature_id|useful_review_count|    sentiment_score|
+--------------------+--------------------+-----+------------+--------------------+--------------------+------------+--------------------+--------------+--------------------+-------------------+-------------------+
|cKyLV5oWZJ2NudWgq...|Oasis Auto Center...|  4.5|          38|edUqROoq7qAVzR2b2...|4oE6BjeHyTVRqd_82...|           2|This place was mu...|9/4/2013 22:11|4oE6BjeHyTVRqd_82...|                  3|0.09166666666666667|
+--------------------+--------------------+-----+------------+--------------------+--------------------+------------+--------------------+--

In [19]:
#finding the average of the sentiment score and the average of the review star ratings
average_sentiment_reviewstarsDF = sentiment_score_df.select("business_id","sentiment_score","review_stars")
from pyspark.sql import functions as F
average_sentiment_reviewstarsDF = average_sentiment_reviewstarsDF.groupBy("business_id").agg({'sentiment_score':'avg', 'review_stars':'avg'})
average_sentiment_reviewstarsDF.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[business_id: string, avg(sentiment_score): double, avg(review_stars): double]

In [20]:
average_sentiment_reviewstarsDF.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----------------+
|         business_id|avg(sentiment_score)|avg(review_stars)|
+--------------------+--------------------+-----------------+
|vyu1WGlnHGv2Z2MVE...|  0.3732122360939325|              5.0|
+--------------------+--------------------+-----------------+
only showing top 1 row

In [21]:
#merging the dataframes 
average_sentiment_reviewstarsDF = average_sentiment_reviewstarsDF.join(sentiment_score_df, "business_id")
average_sentiment_reviewstarsDF.show(1)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----------------+--------------------+-----+------------+--------------------+--------------------+------------+--------------------+--------------+--------------------+-------------------+-------------------+
|         business_id|avg(sentiment_score)|avg(review_stars)|                name|stars|review_count|           review_id|             user_id|review_stars|                text|          date|          feature_id|useful_review_count|    sentiment_score|
+--------------------+--------------------+-----------------+--------------------+-----+------------+--------------------+--------------------+------------+--------------------+--------------+--------------------+-------------------+-------------------+
|cKyLV5oWZJ2NudWgq...|  0.2915578020042854|4.595238095238095|Oasis Auto Center...|  4.5|          38|edUqROoq7qAVzR2b2...|4oE6BjeHyTVRqd_82...|           2|This place was mu...|9/4/2013 22:11|4oE6BjeHyTVRqd_82...|                  3|0.091

In [22]:
#dropping unwanted columns
average_sentiment_reviewstarsDF = average_sentiment_reviewstarsDF.drop("name","stars","review_count")
average_sentiment_reviewstarsDF.show(1)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----------------+--------------------+--------------------+------------+--------------------+--------------+--------------------+-------------------+-------------------+
|         business_id|avg(sentiment_score)|avg(review_stars)|           review_id|             user_id|review_stars|                text|          date|          feature_id|useful_review_count|    sentiment_score|
+--------------------+--------------------+-----------------+--------------------+--------------------+------------+--------------------+--------------+--------------------+-------------------+-------------------+
|cKyLV5oWZJ2NudWgq...|  0.2915578020042854|4.595238095238095|edUqROoq7qAVzR2b2...|4oE6BjeHyTVRqd_82...|           2|This place was mu...|9/4/2013 22:11|4oE6BjeHyTVRqd_82...|                  3|0.09166666666666667|
+--------------------+--------------------+-----------------+--------------------+--------------------+------------+--------------------+-------

In [23]:
 # computing the distance between a single review sentiment score and the average sentiment score and same for review start ratings
from math import *
def compute_mean_distance(mu,data):
     return abs(mu - data)

compute_mean_distance_udf = udf(compute_mean_distance, DoubleType())
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
# Compute distance between Average sentiment score and sentiment score
delta_sentiment_df =  average_sentiment_reviewstarsDF.withColumn("distance_sentiment",compute_mean_distance_udf(average_sentiment_reviewstarsDF["avg(sentiment_score)"],average_sentiment_reviewstarsDF["sentiment_score"]))
delta_sentiment_df.show(1)
                                                                                              

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----------------+--------------------+--------------------+------------+--------------------+--------------+--------------------+-------------------+-------------------+------------------+
|         business_id|avg(sentiment_score)|avg(review_stars)|           review_id|             user_id|review_stars|                text|          date|          feature_id|useful_review_count|    sentiment_score|distance_sentiment|
+--------------------+--------------------+-----------------+--------------------+--------------------+------------+--------------------+--------------+--------------------+-------------------+-------------------+------------------+
|cKyLV5oWZJ2NudWgq...|  0.2915578020042854|4.595238095238095|edUqROoq7qAVzR2b2...|4oE6BjeHyTVRqd_82...|           2|This place was mu...|9/4/2013 22:11|4oE6BjeHyTVRqd_82...|                  3|0.09166666666666667|0.1998911353376187|
+--------------------+--------------------+-----------------+-------

In [25]:
#Compute distance between average review star rating and review stra rating for a single review
delta_review_stars_df =  delta_sentiment_df.withColumn("distance_review_stars",compute_mean_distance_udf(delta_sentiment_df["avg(review_stars)"],delta_sentiment_df["review_stars"]))
delta_review_stars_df.show(1)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----------------+--------------------+--------------------+------------+--------------------+--------------+--------------------+-------------------+-------------------+------------------+---------------------+
|         business_id|avg(sentiment_score)|avg(review_stars)|           review_id|             user_id|review_stars|                text|          date|          feature_id|useful_review_count|    sentiment_score|distance_sentiment|distance_review_stars|
+--------------------+--------------------+-----------------+--------------------+--------------------+------------+--------------------+--------------+--------------------+-------------------+-------------------+------------------+---------------------+
|cKyLV5oWZJ2NudWgq...|  0.2915578020042854|4.595238095238095|edUqROoq7qAVzR2b2...|4oE6BjeHyTVRqd_82...|           2|This place was mu...|9/4/2013 22:11|4oE6BjeHyTVRqd_82...|                  3|0.09166666666666667|0.1998911353376187|   

In [26]:
# Generating features
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

features_assembler_DF = VectorAssembler(
    inputCols=["distance_sentiment", "distance_review_stars", "useful_review_count"],
    outputCol="features")

featuresDF= features_assembler_DF.transform (delta_review_stars_df)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
featuresDF.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----------------+--------------------+--------------------+------------+--------------------+--------------+--------------------+-------------------+-------------------+------------------+---------------------+--------------------+
|         business_id|avg(sentiment_score)|avg(review_stars)|           review_id|             user_id|review_stars|                text|          date|          feature_id|useful_review_count|    sentiment_score|distance_sentiment|distance_review_stars|            features|
+--------------------+--------------------+-----------------+--------------------+--------------------+------------+--------------------+--------------+--------------------+-------------------+-------------------+------------------+---------------------+--------------------+
|cKyLV5oWZJ2NudWgq...|  0.2915578020042854|4.595238095238095|edUqROoq7qAVzR2b2...|4oE6BjeHyTVRqd_82...|           2|This place was mu...|9/4/2013 22:11|4oE6BjeHyTVRqd_82...

In [28]:
# Standardising using MaxAbsScalar
from pyspark.ml.feature import MaxAbsScaler
from pyspark.ml.linalg import Vectors

scaler = MaxAbsScaler(inputCol="features", outputCol="scaledFeatures")
# Compute summary statistics and generate MaxAbsScalerModel
scalerModel = scaler.fit(featuresDF)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
scaledTransformedData = scalerModel.transform(featuresDF)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
# creating empty dataframe so all to append final results.
schema = StructType([
    StructField("Silhouette_distance", DoubleType(), True),
    StructField("k", IntegerType(), True)        
])
K_Silhouette_DF = spark.createDataFrame(sc.emptyRDD(), schema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
from pyspark.ml.clustering import GaussianMixture

def create_GMM_model(k, scaledTransformedDF):
    gmm = GaussianMixture().setK(k).setFeaturesCol("scaledFeatures").setMaxIter(100)
    model = gmm.fit(scaledTransformedDF)
    return model


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
# Write to CSV File
def toCSV(df,outputPath):
    df.write.csv(outputPath)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
from pyspark.ml.evaluation import ClusteringEvaluator


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
#checking the probability and labelling it as spam review or ham review
def probability_distribution_scan(probability):
    return probability.toArray().tolist()[0] >= 0.90

probability_distribution_scan_udf = udf(probability_distribution_scan,  BooleanType())


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
#iterating the model on different k values
k=2
s_distance = []
while (k <= 15) :
    gmm = create_GMM_model(k,scaledTransformedData)
    
    transformed_gmm = gmm.transform(scaledTransformedData)
    
    cosine_evaluator = ClusteringEvaluator().setDistanceMeasure ("cosine")
    
    Silhouette_distance = cosine_evaluator.evaluate(transformed_gmm);
    
    tuple_row = spark.createDataFrame([{"k": k, "Silhouette_distance": Silhouette_distance}])
    
    K_Silhouette_DF = K_Silhouette_DF.union(tuple_row)
    
    s_distance.append(Silhouette_distance)
    
    reviewDF = transformed_gmm.withColumn("isHam",probability_distribution_scan_udf(transformed_gmm["probability"]))
    
    newReviewDF = reviewDF.select("business_id","user_id","review_id","prediction","isHam")
    
    toCSV(newReviewDF,"s3://reviwssannilbhav/out/k_new"+str(k))
    
    if k== 2:
        K_Silhouette_DF.show(1)
    
    k = k+1;

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

silhouette width 0.13103409943453684 for K 2
+-------------------+---+
|Silhouette_distance|  k|
+-------------------+---+
|0.13103409943453684|  2|
+-------------------+---+
only showing top 1 row

/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead

In [48]:
K_Silhouette_DF.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---+
| Silhouette_distance|  k|
+--------------------+---+
| 0.13103409943453684|  2|
| 0.13103409943453684|  2|
|-0.06918256577999014|  3|
|-0.04822272141379496|  4|
+--------------------+---+

In [43]:
toCSV(K_Silhouette_DF,"s3://reviwssannilbhav/out/s_diatance_new") 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'path s3://reviwssannilbhav/out/s_diatance_new already exists.;'
Traceback (most recent call last):
  File "<stdin>", line 3, in toCSV
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/readwriter.py", line 931, in csv
    self._jwrite.csv(path)
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: 'path s3://reviwssannilbhav/out/s_diatance_new already exists.;'

